In [29]:
# !pip install --upgrade jax[cuda] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [13]:
# using Kernel PyTorch-2.2.0

import os
import csv
import jax
import haiku as hk
import numpy as np
import pandas as pd
import pandas as pd

# import torch
# import pickle
# import swifter
# import seaborn as sns
# from tqdm import tqdm

import jax.numpy as jnp
import matplotlib.pyplot as plt

import multiprocessing as mp

from nucleotide_transformer.pretrained import get_pretrained_model

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

from datasets import Dataset, DatasetDict


%run preprocess_utility.py

print(jax.devices())



# import os
# import jax
# # import torch
# # import pickle
# import pandas as pd
# import haiku as hk
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import jax.numpy as jnp
# import matplotlib.pyplot as plt

# from tqdm import tqdm
# from sklearn.preprocessing import StandardScaler
# from transformers import AutoModel #, AutoModelForMaskedLM
# from nucleotide_transformer.pretrained import get_pretrained_model

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

[CudaDevice(id=0)]


In [14]:
try:
    import nucleotide_transformer
except:
    !pip install numpy==1.23.5
    !pip install git+https://github.com/instadeepai/nucleotide-transformer@main |tail -n 1
    import nucleotide_transformer

if "COLAB_TPU_ADDR" in os.environ:
    from jax.tools import colab_tpu

    colab_tpu.setup_tpu()

In [15]:
comp = {'A':1, 'C':2, 'G':3, 'T':4}

#@title Select a model
#@markdown ---
model_name = '50M_multi_species_v2'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
model_name = '500M_human_ref'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
#@markdown ---

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    embeddings_layers_to_save=(20,),
    attention_maps_to_save=((1, 4), (7, 18)),
    max_positions=32,
    # If the progress bar gets stuck at the start of the model wieghts download,
    # you can set verbose=False to download without the progress bar.
    verbose=False
)
forward_fn = hk.transform(forward_fn)

In [16]:
# Get data and tokenize it
# sequences = [
#     "ATTCCGAAATCGCTGACCGATCGTACGAAA",
#     "ATTTCTCTCTCTCTCTGAGATCGATCGATCGATATCTCTCGAGCTAGC",
# ]
# tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
# tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
# tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)

In [17]:
# !pip install jax jaxlib==0.1.87+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [18]:
# import jax
# import jax.numpy as jnp

# x = jnp.ones((1000, 1000))
# y = jnp.dot(x, x.T).block_until_ready()

# %%time
# # Initialize random key
# random_key = jax.random.PRNGKey(0)

# # Infer
# outs = forward_fn.apply(parameters, random_key, tokens)

In [19]:
fasta_file = "/blue/xiaofan/sunhuaikuan/gpn/examples/msa/genome.hg38rg.fa"
chrom_sequences = read_fasta(fasta_file)

In [20]:
def get_subsequence(sequences, chrom_name, start_pos, length):
    
    if chrom_name in sequences:
        sequence = sequences[chrom_name]
        subsequence = sequence[start_pos:start_pos + length]
        return subsequence
    else:
        raise ValueError(f"Chromosome '{chrom_name}' not found in the FASTA file.")

In [21]:
def append_data(final_df, sub_df, sub_embedding_df):
    
    sub_df = sub_df.reset_index(drop=True)
    sub_embedding_df = sub_embedding_df.reset_index(drop=True)
    
    sub_final_df = pd.concat([sub_embedding_df, sub_df],  axis=1, ignore_index=True)
    final_df = pd.concat([final_df, sub_final_df],  axis=0, ignore_index=True) 
    
    return final_df

In [22]:
def get_tokens(df):
    sequences = []
    # refs=[]
    for index, row in df.iterrows():      
        chrom=row['CHROM']
        # pos_start=row['POS']-93
        pos_start=row['START']
        # ref=row['REF']
        # alt=row['ALT']        
        rowid=row['ROWID']
        y=row['y']
        if row['SIZE'] % 6 == 0:
            length = row['SIZE']
        else:
            length = 6 * round(row['SIZE'] / 6)
    
        subsequence = get_subsequence(chrom_sequences, chrom, pos_start, length)
        if 'N' in subsequence:
            print("The character 'N' is present in the string.")
    
        sequences.append(subsequence)
        # refs.append(ref)
        # print(f"Subsequence from {chrom_name} starting at position {start_pos}:\n{subsequence}")
    
    print (len(sequences))
    # sequences

    try:
        tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
        tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
        tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)   

    except Exception as e:
        print(f"exception caught: {e}"+str(row['CHROM'])+'-'+str(row['START'])+'-'+str(row['SIZE']))
        # print(f"exception caught: {e}")
        tokens=None
        
    return tokens

In [23]:
def get_embeddings(tokens):

    # Initialize random key
    random_key = jax.random.PRNGKey(0)
    
    # Infer
    outs = forward_fn.apply(parameters, random_key, tokens)    
    # print(outs.keys())
    # print(outs["embeddings_20"].shape)
    
    
    # my_embedding=outs["embeddings_20"][:,16,:]
    my_embedding=outs["embeddings_20"][:,0,:]
    
    my_embedding.shape
    column_names = [f'{i}' for i in range(0, my_embedding.shape[1])]
    embedding_df = pd.DataFrame(my_embedding, columns=column_names)
    return embedding_df

### Process datafile 

In [24]:
max_length= 186

datafile_path = '/blue/xiaofan/chenyuanhan/data/Homo_sapiens.GRCh38.109.txt.gz'  
df = preprocess_home_sapiens_datafile(datafile_path)

df.loc[df['SIZE'] > max_length, 'END'] = df['START'] + max_length
df.loc[df['SIZE'] > max_length, 'SIZE'] = max_length

df.drop(columns=['END','TYPE','CLUSTER'], inplace=True)

df

,CHROM,START,SIZE,ROWID,y
0,7,116953541,150,0,0
1,12,54241755,150,1,0
2,20,38033948,150,2,0
3,15,82659560,150,3,0
4,9,131579497,150,4,0
...,...,...,...,...,...
34832,17,19189665,186,34832,6
34833,17,19188016,186,34833,6
34834,17,19557211,186,34834,6
34835,17,19061912,186,34835,6


In [25]:
%%time

sub_df = pd.DataFrame()    
final_df = pd.DataFrame()
segment=800


csv_Filename = './homo_sapiens_nt_embedding.csv'
if os.path.exists(csv_Filename):
    os.remove(csv_Filename)


cnt=0
for index, row in df.iterrows():
    cnt+=1
    sub_df = sub_df.drop(sub_df.index)
    
    if (cnt % segment==0):
        sub_df = df.iloc[cnt-segment:cnt]
        sub_tokens = get_tokens(sub_df)
        sub_embedding_df = get_embeddings(sub_tokens)       
        final_df = append_data(final_df, sub_df, sub_embedding_df)
        
        sub_df = sub_df.reset_index(drop=True)
        print(f"complete batch...... {cnt}")


print(f"last index...... {(cnt)}")
sub_df = df.iloc[cnt-(cnt % segment):cnt]
sub_tokens = get_tokens(sub_df)
sub_embedding_df = get_embeddings(sub_tokens)        
final_df = append_data(final_df, sub_df, sub_embedding_df)


column_names = [f'{i}' for i in range(0, final_df.shape[1]-5)]
column_names.extend(['CHROM', 'START', 'SIZE', 'ROWID',  'y'])
final_df.columns = column_names
final_df = final_df.drop(columns=['CHROM','START','SIZE'])

final_df.to_csv(csv_Filename, sep=',', index=False,  header=True, na_rep='NaN')

final_df

800


2024-11-01 11:44:51.531758: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Before attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-0.12643832  0.8992672  -2.0135226  ... -0.77297044  0.0352236
    0.16593635]
  [-0.15411776  0.65062594  0.03419471 ...  0.10655487 -0.7579897
   -1.9504462 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]]

 [[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-2.7441926   0.48109138 -0.6968985  ...  0.55498445 -0.17344053
   -0.36260185]
  [-1.3409749   0.749942    1.5137826  ...  0.5966591   0.6111201
   -0.8521586 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -

,0,1,2,3,4,5,6,7,8,9,...,1272,1273,1274,1275,1276,1277,1278,1279,ROWID,y
0,-0.324386,10.222736,0.472543,8.665915,-9.160305,3.684612,-9.674951,9.519714,1.577347,39.271915,...,-8.166821,-4.011395,6.786868,-5.835679,-18.031162,-12.672304,-4.555680,0.773324,0,0
1,0.215155,13.518826,3.466785,9.049223,-9.875353,-0.560636,-9.800148,9.256715,1.785120,38.618298,...,-5.242308,-1.541120,4.458237,-3.372420,-20.867760,-16.494043,-8.260738,3.139253,1,0
2,-2.372187,13.168259,2.926776,11.876443,-8.724695,-0.121530,-5.909375,7.047293,3.658315,40.302120,...,-5.853244,-3.642528,6.743409,-3.631339,-20.687992,-14.483265,-3.643872,5.543061,2,0
3,-1.159037,17.319054,1.178315,5.559775,-7.308437,3.267867,-8.044428,12.884748,2.865854,38.994896,...,-6.229579,2.583466,3.808581,-1.381266,-22.413837,-11.401751,-0.634754,1.241418,3,0
4,-0.237370,14.949961,2.041708,7.604409,-8.435667,-2.641743,-9.003962,9.609196,2.431862,44.436890,...,-5.326154,-0.373121,6.259539,-1.734646,-21.528700,-13.961947,-7.830993,-0.421597,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34786,0.592047,12.480164,-2.207586,8.692019,-5.893743,7.420892,-12.051699,13.012270,3.701627,38.131466,...,-2.668526,-5.377493,3.924320,-3.992713,-20.384758,-14.261722,-1.868958,-1.269481,34832,6
34787,-1.070907,10.931201,3.266546,8.628684,-5.165432,6.097012,-6.288345,11.052012,5.249962,39.650799,...,-3.123629,0.197064,4.605883,-1.002314,-21.945362,-11.409150,-4.581974,2.272008,34833,6
34788,-0.759760,9.573089,7.653216,-0.243169,-8.327825,6.982180,-5.615465,10.963967,4.588292,42.244381,...,-6.250230,-1.333322,2.300076,-2.416507,-21.926449,-10.241255,-3.423506,1.949625,34834,6
34789,-0.543252,11.539721,3.129791,9.273889,-4.417173,5.724512,-6.320546,11.377898,4.870200,39.562275,...,-3.272681,0.535542,4.866612,-0.382214,-21.396807,-11.046916,-5.265911,2.063335,34835,6


### Load CSV File

In [26]:
def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    # print(df.shape)
    # # new_arr = data_array[:, :-4]
    # # last_column = data_array[:, -1]
    # # data_array = np.column_stack((new_arr, last_column))
    
    # column_names = [f'{i}' for i in range(0, df.shape[1]-5)]
    # column_names.extend(['CHROM', 'START', 'SIZE', 'ROWID',  'y'])
    
    # df.columns = column_names
    return df

df = load_embedding_file('./homo_sapiens_nt_embedding.csv')
# df = load_embedding_file('/blue/xiaofan/sunhuaikuan/PAPERS/DNA_LLM_REVIEW/embeddings/homo-sapiens/nt/Homo_sapiens_embeddings_nt.csv')

df

,0,1,2,3,4,5,6,7,8,9,...,1272,1273,1274,1275,1276,1277,1278,1279,ROWID,y
0,-0.324386,10.222736,0.472543,8.665916,-9.160305,3.684612,-9.674951,9.519714,1.577347,39.271915,...,-8.166821,-4.011395,6.786868,-5.835679,-18.031162,-12.672304,-4.555680,0.773324,0,0
1,0.215155,13.518826,3.466785,9.049223,-9.875353,-0.560636,-9.800148,9.256715,1.785120,38.618298,...,-5.242308,-1.541120,4.458237,-3.372420,-20.867760,-16.494043,-8.260738,3.139253,1,0
2,-2.372187,13.168259,2.926776,11.876443,-8.724695,-0.121530,-5.909375,7.047293,3.658315,40.302120,...,-5.853244,-3.642528,6.743409,-3.631339,-20.687992,-14.483265,-3.643872,5.543061,2,0
3,-1.159037,17.319054,1.178315,5.559775,-7.308437,3.267867,-8.044428,12.884747,2.865854,38.994896,...,-6.229579,2.583466,3.808581,-1.381266,-22.413837,-11.401751,-0.634754,1.241418,3,0
4,-0.237370,14.949961,2.041708,7.604409,-8.435667,-2.641743,-9.003962,9.609196,2.431862,44.436890,...,-5.326154,-0.373121,6.259539,-1.734646,-21.528700,-13.961947,-7.830993,-0.421597,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34786,0.592047,12.480164,-2.207586,8.692018,-5.893743,7.420892,-12.051699,13.012270,3.701627,38.131466,...,-2.668526,-5.377493,3.924320,-3.992713,-20.384758,-14.261722,-1.868958,-1.269481,34832,6
34787,-1.070907,10.931201,3.266546,8.628684,-5.165432,6.097012,-6.288345,11.052012,5.249962,39.650800,...,-3.123629,0.197064,4.605883,-1.002314,-21.945362,-11.409150,-4.581974,2.272008,34833,6
34788,-0.759760,9.573089,7.653216,-0.243169,-8.327825,6.982180,-5.615465,10.963967,4.588292,42.244380,...,-6.250230,-1.333322,2.300076,-2.416507,-21.926449,-10.241255,-3.423506,1.949625,34834,6
34789,-0.543251,11.539721,3.129790,9.273889,-4.417173,5.724512,-6.320546,11.377898,4.870200,39.562275,...,-3.272682,0.535543,4.866612,-0.382214,-21.396807,-11.046916,-5.265911,2.063335,34835,6
